In [4]:
from IPython.display import display, Math, Latex



AdaNet: Adaptive Structural Learning of Neural Networks [Cortes et al., ICML 2017] defines an algorithm that aims to directly minimize the DeepBoost generalization bound from Deep Boosting [Cortes et al., ICML 2014] whenapplied to neural networks. This algorithm, called AdaNet, adaptively grows a neural network as an ensemble ofsubnetworks that minimizes the AdaNet objective (a.k.a. AdaNet loss):

$$F(w) = \frac{1}{m} \sum_{i=1}^{m} \Phi \left(\sum_{j=1}^{N}w_jh_j(x_i), y_i \right) + \sum_{j=1}^{N} \left(\lambda r(h_j) + \beta \right) |w_j| $$

where $w$ is the set of mixture weights, one per subnetwork $h$, $\Phi$ is a surrogate loss function such as logistic loss or MSE, $r$ is a function for measuring a subnetwork's complexity, and $\lambda$ and $\beta$ are hyperparameters.


How AdaNet uses the objective

This objective function serves two purposes:
To learn to scale/transform the outputs of each subnetwork $h$ as part of the ensemble.
To select the best candidate subnetwork $h$ at each AdaNet iteration to include in the ensemble.
Effectively, when learning mixture weights $w$ AdaNet solves a convex combination of the outputs of the frozen subnetworks $h$. For $$\lambda \gt 0,$$ AdaNet penalizes more complex subnetworks with greater L1 regularization on their mixture weight, and will be less 
likely to select more complex subnetworks to add to the ensemble at each iteration.


We will solve a regression task known as the Boston Housing dataset to predict the price of suburban houses in Boston. There are $13$ numerical features, the labels are in thousands of dollars, and there are only 506 examples.



In [5]:
# We run experiments to analyze the potential advantages of updating the architecture of a Deep Neural Network during
# optimization, and simultaneously using as solution an ensemble of weighted networks (see below).  It is crucial to
# observe that during training we obtain:

# -The optimum parametrization of the network (as in the classical sense).
# -The optimal architecture of the network, which is variable during runtime.
# -A set of "ensemble" weights that minimizes the objective function.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import trainAndEvaluate
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)

(x_train, y_train), (x_test, y_test) = (
    tf.keras.datasets.boston_housing.load_data())

# @title AdaNet parameters
LEARNING_RATE = 0.001   # @param {type:"number"}
TRAIN_STEPS = 1000      # @param {type:"integer"} 100000
BATCH_SIZE = 32         # @param {type:"integer"}


# Use variable weights that appear in linear combination of subnetworks.
# LEARN_MIXTURE_WEIGHTS # @param {type:"boolean"}

# lambda parameter serves to prevent the optimization from assigning too
# much weight to more complex subnetworks
# ADANET_LAMBDA         # @param {type:"number"}

# Size of adaptive architecture of AdamNet.
# BOOSTING_ITERATIONS = 5  # @param {type:"integer"}

loss_results = []
for i in range(1, 4):
    results, _ = trainAndEvaluate.train_and_evaluate(x_train, y_train, x_test, y_test,
                                                     LEARNING_RATE,
                                                     TRAIN_STEPS,
                                                     BATCH_SIZE,
                                                     learn_mixture_weights=False,
                                                     adanet_lambda=0,
                                                     boosting_iterations=i)
    loss_results.append(results["average_loss"])
    print(loss_results)
    print("Loss:", results["average_loss"])
    print("Architecture:", trainAndEvaluate.ensemble_architecture(results))

plt.scatter(list(range(1, 4)), loss_results)
plt.show()


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/84/qt8t4z6j35dfzq394ljwcqdw0000gn/T/tmpc85ld34q', '_tf_random_seed': 42, '_save_summary_steps': 50000, '_save_checkpoints_steps': 50000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f9da240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 50000 or save_checkpoints_secs None.


INFO:tensorflow:Beginning training AdaNet iteration 0


...

INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.08889891, average_loss/adanet/adanet_weighted_ensemble = 0.08889891, average_loss/adanet/subnetwork = 0.093816034, average_loss/adanet/uniform_average_ensemble = 0.08889893, global_step = 1000, label/mean = 3.1049454, label/mean/adanet/adanet_weighted_ensemble = 3.1049454, label/mean/adanet/subnetwork = 3.1049454, label/mean/adanet/uniform_average_ensemble = 3.1049454, loss = 0.09088868, loss/adanet/adanet_weighted_ensemble = 0.09088868, loss/adanet/subnetwork = 0.092960946, loss/adanet/uniform_average_ensemble = 0.0908887, prediction/mean = 2.9723778, prediction/mean/adanet/adanet_weighted_ensemble = 2.9723778, prediction/mean/adanet/subnetwork = 2.9756782, prediction/mean/adanet/uniform_average_ensemble = 2.972378


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /var/folders/84/qt8t4z6j35dfzq394ljwcqdw0000gn/T/tmpq_pj8_m8/model.ckpt-1000


INFO:tensorflow:Loss for final step: 0.06004364.


[0.07330291, 0.06450059, 0.08889891]
Loss: 0.08889891
Architecture: b"| b'1_layer_dnn' | b'2_layer_dnn' | b'2_layer_dnn' |"


<Figure size 640x480 with 1 Axes>